# ML to predict Bitcoin prices

+ Data from: https://coinmarketcap.com/api/
+ Adapted from: https://dashee87.github.io/data%20science/deep%20learning/python/another-keras-tutorial-for-neural-network-beginners/
+ Adapted from: https://dashee87.github.io/deep%20learning/python/predicting-cryptocurrency-prices-with-deep-learning/
+ Adapted from: https://github.com/dashee87/blogScripts/blob/master/Jupyter/2017-11-20-predicting-cryptocurrency-prices-with-deep-learning.ipynb

In [1]:
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

# get market info for bitcoin from the start of 2016 to the current day
bitcoin_market_info = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]

# convert the date string to the correct date format
bitcoin_market_info = bitcoin_market_info.assign(Date=pd.to_datetime(bitcoin_market_info['Date']))
# when Volume is equal to '-' convert it to 0
bitcoin_market_info.loc[bitcoin_market_info['Volume']=="-",'Volume']=0
# convert to int
bitcoin_market_info['Volume'] = bitcoin_market_info['Volume'].astype('int64')
# look at the first few rows
bitcoin_market_info.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,2018-04-14,7874.67,8140.71,7846.00,7986.24,5191430000,133682000000
1,2018-04-13,7901.09,8183.96,7758.93,7895.96,7764460000,134114000000
2,2018-04-12,6955.38,7899.23,6806.51,7889.25,8906250000,118048000000
3,2018-04-11,6843.47,6968.32,6817.59,6968.32,4641890000,116126000000
4,2018-04-10,6795.44,6872.41,6704.15,6834.76,4272750000,115306000000


In [2]:
bitcoin_market_info.columns =[bitcoin_market_info.columns[0]]+['bt_'+i for i in bitcoin_market_info.columns[1:]]

In [3]:
# market_info = pd.merge(bitcoin_market_info,eth_market_info, on=['Date'])
# market_info = market_info[market_info['Date']>='2016-01-01']
market_info = bitcoin_market_info
market_info = market_info[market_info['Date']>='2016-01-01']
for coins in ['bt_']: 
    kwargs = { coins+'day_diff': lambda x: (x[coins+'Close']-x[coins+'Open'])/x[coins+'Open']}
    market_info = market_info.assign(**kwargs)
market_info.head()

,Date,bt_Open,bt_High,bt_Low,bt_Close,bt_Volume,bt_Market Cap,bt_day_diff
0,2018-04-14,7874.67,8140.71,7846.00,7986.24,5191430000,133682000000,0.014168
1,2018-04-13,7901.09,8183.96,7758.93,7895.96,7764460000,134114000000,-0.000649
2,2018-04-12,6955.38,7899.23,6806.51,7889.25,8906250000,118048000000,0.134266
3,2018-04-11,6843.47,6968.32,6817.59,6968.32,4641890000,116126000000,0.018244
4,2018-04-10,6795.44,6872.41,6704.15,6834.76,4272750000,115306000000,0.005786


In [4]:
for coins in ['bt_']: 
    kwargs = { coins+'close_off_high': lambda x: 2*(x[coins+'High']- x[coins+'Close'])/(x[coins+'High']-x[coins+'Low'])-1,
            coins+'volatility': lambda x: (x[coins+'High']- x[coins+'Low'])/(x[coins+'Open'])}
    market_info = market_info.assign(**kwargs)

In [5]:
model_data = market_info[['Date']+[coin+metric for coin in ['bt_'] 
                                   for metric in ['Close','Volume','close_off_high','volatility']]]
# need to reverse the data frame so that subsequent rows represent later timepoints
model_data = model_data.sort_values(by='Date')
model_data.head()

,Date,bt_Close,bt_Volume,bt_close_off_high,bt_volatility
834,2016-01-01,434.33,36278900,-0.560641,0.020292
833,2016-01-02,433.44,30096600,0.250597,0.009641
832,2016-01-03,430.01,39633800,-0.173865,0.020827
831,2016-01-04,433.09,38477500,-0.474265,0.012649
830,2016-01-05,431.96,34522600,-0.013333,0.010391


In [6]:
split_date = '2017-06-01'
# we don't need the date columns anymore
training_set, test_set = model_data[model_data['Date']<split_date], model_data[model_data['Date']>=split_date]
training_set = training_set.drop('Date', 1)
test_set = test_set.drop('Date', 1)

In [7]:
window_len = 1
norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume']]

In [8]:
LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_training_inputs.append(temp_set)
LSTM_training_outputs = (training_set['bt_Close'][window_len:].values/training_set['bt_Close'][:-window_len].values)-1

In [9]:
LSTM_training_inputs[0]

,bt_Close,bt_Volume,bt_close_off_high,bt_volatility
834,0.0,0.0,-0.560641,0.020292


In [10]:
LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_test_inputs.append(temp_set)
LSTM_test_outputs = (test_set['bt_Close'][window_len:].values/test_set['bt_Close'][:-window_len].values)-1

In [11]:
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [12]:
# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

Using TensorFlow backend.


In [13]:
# random seed for reproducibility
np.random.seed(202)
# we'll try to predict the closing price for the next 5 days 
# change this value if you want to make longer/shorter prediction
pred_range = 1
# initialise model architecture
bt_model = build_model(LSTM_training_inputs, output_size=pred_range, neurons = 20)
# model output is next 5 prices normalised to 10th previous closing price
LSTM_training_outputs = []
for i in range(window_len, len(training_set['bt_Close'])-pred_range):
    LSTM_training_outputs.append((training_set['bt_Close'][i:i+pred_range].values/
                                  training_set['bt_Close'].values[i-window_len])-1)
LSTM_training_outputs = np.array(LSTM_training_outputs)
# train model on data
# note: eth_history contains information on the training error per epoch
bt_history = bt_model.fit(LSTM_training_inputs[:-pred_range], LSTM_training_outputs, 
                            epochs=50, batch_size=1, verbose=2, shuffle=True)

Epoch 1/50
 - 2s - loss: 0.0237
Epoch 2/50
 - 1s - loss: 0.0221
Epoch 3/50
 - 1s - loss: 0.0212
Epoch 4/50
 - 1s - loss: 0.0206
Epoch 5/50
 - 1s - loss: 0.0197
Epoch 6/50
 - 1s - loss: 0.0193
Epoch 7/50
 - 1s - loss: 0.0189
Epoch 8/50
 - 1s - loss: 0.0190
Epoch 9/50
 - 1s - loss: 0.0191
Epoch 10/50
 - 1s - loss: 0.0190
Epoch 11/50
 - 1s - loss: 0.0189
Epoch 12/50
 - 1s - loss: 0.0187
Epoch 13/50
 - 1s - loss: 0.0188
Epoch 14/50
 - 1s - loss: 0.0188
Epoch 15/50
 - 1s - loss: 0.0187
Epoch 16/50
 - 1s - loss: 0.0186
Epoch 17/50
 - 1s - loss: 0.0187
Epoch 18/50
 - 1s - loss: 0.0188
Epoch 19/50
 - 1s - loss: 0.0186
Epoch 20/50
 - 1s - loss: 0.0185
Epoch 21/50
 - 1s - loss: 0.0187
Epoch 22/50
 - 1s - loss: 0.0187
Epoch 23/50
 - 1s - loss: 0.0187
Epoch 24/50
 - 1s - loss: 0.0187
Epoch 25/50
 - 1s - loss: 0.0187
Epoch 26/50
 - 1s - loss: 0.0186
Epoch 27/50
 - 1s - loss: 0.0186
Epoch 28/50
 - 1s - loss: 0.0185
Epoch 29/50
 - 1s - loss: 0.0187
Epoch 30/50
 - 1s - loss: 0.0185
Epoch 31/50
 - 1s -

In [14]:
bt_pred_prices = ((bt_model.predict(LSTM_test_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)][::pred_range].reshape(int(np.ceil((len(LSTM_test_inputs)-pred_range)/float(pred_range))),1))

In [15]:
bt_pred_prices

array([[  2420.73343868],
       [  2498.52513458],
       [  2529.93665324],
       [  2526.51745606],
       [  2698.96094459],
       [  2887.60165215],
       [  2753.85870895],
       [  2817.53599719],
       [  2840.15378709],
       [  2961.30334155],
       [  2971.96076577],
       [  2698.51967226],
       [  2732.35864929],
       [  2540.1587252 ],
       [  2482.55032782],
       [  2530.37848   ],
       [  2669.81064499],
       [  2567.11280648],
       [  2604.64810352],
       [  2735.88076406],
       [  2705.47417542],
       [  2715.70484184],
       [  2754.87544767],
       [  2631.40781276],
       [  2606.41095477],
       [  2499.20684384],
       [  2565.06892395],
       [  2586.23513721],
       [  2551.56797088],
       [  2497.36353912],
       [  2450.23876551],
       [  2516.8396634 ],
       [  2575.5314112 ],
       [  2612.18474831],
       [  2612.16550015],
       [  2616.92836793],
       [  2565.96492103],
       [  2580.67070059],
       [  25

In [16]:
test = np.array([[[0.0,0.0,-0.560641,0.020292]]])

In [17]:
bt_pred_prices = bt_model.predict(test)

In [18]:
bt_pred_price

NameError: name 'bt_pred_price' is not defined

In [ ]:
test_price = (bt_model.predict(LSTM_test_inputs))

Querying like above, without the full calculation, returns wrong results. Calculation:

```
bt_pred_prices = ((bt_model.predict(LSTM_test_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)[::pred_range].reshape(int(np.ceil((len(LSTM_test_inputs)-pred_range)/float(pred_range))),1))
```
Think the solution into read in and prepare data (test_set, LSTM_test_inputs) from scratch using only one days worth of data.

In [ ]:
test_price